## Mathsolve - Photomath demo project

The goal of this project is to make a model which would be able to detect handwritten math symbols on a picture. After detecting math symbols the app should be able to solve simple mathematical problem given on a picture (addition, subtraction, multiplication and divison).

### Data augmentation

In [74]:
# import useful libraries
import cv2 as cv
import numpy as np
import pandas as pd
import os
import PIL
import pathlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# import tensorflow and keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [60]:
# path to the main dataset directory
dataset_main_path = r"C:\Users\Korisnik\git_workspace\MathSolve\dataset"

In [61]:
# dataset path object
dataset_path_object = pathlib.Path(dataset_main_path)

In [62]:
# dictionary with class names and all associated image paths
image_paths_dict = {}

# list of casses present in the dataset
dataset_classes = os.listdir(dataset_main_path)

for image_class in dataset_classes:
    image_paths_dict[image_class]=[image_path for image_path in dataset_path_object.glob('{}/*'.format(image_class))]

In [63]:
# check 
image_paths_dict

{'0': [WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/0CdBlhLw.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/10014.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/1058.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/10679.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/10825.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/10892.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/10941.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/10970.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/1199.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/12488.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/12580.jpg'),
  WindowsPath('C:/Users/Korisnik/git_workspace/MathSolve/dataset/0/12643.jpg'),
  WindowsPath('C:/Users/Korisnik/g

Now that we have all image paths associated with classes, we need to assign labels to each class.

In [64]:
# class labels dictionary
class_labels_dict = dict()

i = 0

for image_class in dataset_classes:
    class_labels_dict[image_class] = i
    i+=1

In [65]:
# check
class_labels_dict

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'add': 10,
 'dec': 11,
 'div': 12,
 'eq': 13,
 'mul': 14,
 'sub': 15,
 'x': 16,
 'y': 17,
 'z': 18}

Let's read first image of class 0 into numpy array with help of opencv.

In [66]:
img = cv.imread(str(image_paths_dict['0'][0]))

img

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [67]:
# check shape
img.shape

(400, 400, 3)

We need to resize all images to some standard shape. Dataset contains some images 400x400 pixels, and some 155x155 pixels, but we need all images to be the same shape (model expects all samples to be the same dimension). We'll write function that will loop through all images and resize them to 155x155 pixels. 

In [68]:
def resize_images(dataset, labels, pixels_height, pixels_width):
    """This function resizes all images in the dataset to desired pixels height and pixels width.
    
    Args:
        dataset (dict): Dictionary with class names and all associated image paths.
        labels (dict): Dictionary with class names associated with corresponding labels 
        pixels_height (int): Desired height of the resized image in pixels.
        pixels_height (int): Desired width of the resized image in pixels.
    Returns:
        X (list): List of resized images.
        y (list): List of labels corresponding to list of.
    """
    X, y = [], []
    
    for class_name, images in dataset.items():
        for image in images:
                img = cv.imread(str(image))
                try:
                    resized_image = cv.resize(img, (pixels_height, pixels_width))
                    X.append(resized_image)
                    y.append(labels[class_name])
                except:
                    break
    return X, y

In [69]:
# create dataset prepared for train test split
X, y = resize_images(image_paths_dict, class_labels_dict, 150, 150)

In [71]:
len(X), len(y)

(10071, 10071)

In [72]:
X[0]

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [73]:
# convert python list to numpy array
X = np.array(X)
y = np.array(y)